# Inspecting geometry information
We often would like to check the geometry of our simulation: the size of a volume or its material, for example. Information concerning the geometry is saved along the simulation output, in the `geofile`. FairShip itself provides a macro to read this file: $FAIRSHIP/macro/getGeoInformation.py. I have copied the script in this folder to run it together and load the geometry

## Output of the script
Launching it with only the -g option (requiring the geofile), will print only the information of the mother volumes, the one directly below the top volume:

In [1]:
# we need to copy the geometry file from EOS here.
inputgeofile = "/eos/user/a/aiuliano/public/sims_FairShip/sim_snd/numuCCDIS_10_11_2020/geofile_full.conical.Genie-TGeant4.root"
!cp $inputgeofile ./

In [2]:
run getGeoInformation.py -g geofile_full.conical.Genie-TGeant4.root

Welcome to JupyROOT 6.20/06
   Detector element             z(midpoint)     halflength       volume-start volume-end   dx                x-start       x-end       dy                y-start       y-end         material
volTarget_1                 : z=    0.0000cm  dZ=   27.4888cm  [  -27.4888      27.4888] dx=   20.5000cm [    8.0000      49.0000] dy=   20.5000cm [   15.5000      56.5000]                  air
floor_1                     : z=   80.0000cm  dZ=  130.0000cm  [  -50.0000     210.0000] dx=   50.0000cm [  -20.0000      80.0000] dy=   34.0000cm [  -60.0000       8.0000]             Concrete
volMuFilter_1               : z=  118.4888cm  dZ=   91.0000cm  [   27.4888     209.4888] dx=   31.0000cm [   -2.5000      59.5000] dy=   33.2500cm [    3.5500      70.0500]                  air


We can see the half dimensions, position and material of the volume. We can insect the daughters of a specific volume by adding the option `-v volumname -l daugtherlevel`:

In [3]:
run getGeoInformation.py -g geofile_full.conical.Genie-TGeant4.root -v MuonFilter_1 -l 1

   Detector element             z(midpoint)     halflength       volume-start volume-end   dx                x-start       x-end       dy                y-start       y-end         material
volTarget_1                 : z=    0.0000cm  dZ=   27.4888cm  [  -27.4888      27.4888] dx=   20.5000cm [    8.0000      49.0000] dy=   20.5000cm [   15.5000      56.5000]                  air
floor_1                     : z=   80.0000cm  dZ=  130.0000cm  [  -50.0000     210.0000] dx=   50.0000cm [  -20.0000      80.0000] dy=   34.0000cm [  -60.0000       8.0000]             Concrete
volMuFilter_1               : z=  118.4888cm  dZ=   91.0000cm  [   27.4888     209.4888] dx=   31.0000cm [   -2.5000      59.5000] dy=   33.2500cm [    3.5500      70.0500]                  air


Warning in <TGeoManager::Init>: Deleting previous geometry: FAIRGeom/FAIR geometry
Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 154
Info in <TGeoManager::CloseGeometry>: 1113 nodes/ 15 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------


## Accesing information about a specific volume
Of course, finding a specific volume in this way is quite time-consuming, especially if our desired volume is many levels below the top volume. However, we can directly get our volume by his name thanks to the gGeoManager

In [4]:
volTarget = fGeo.GetVolume("volTarget")
volTarget

<ROOT.TGeoVolume object ("volTarget") at 0x6440340>

For example, let us ask the half-sizes of our shapes. Remind: different shapes have different size definitions. Vol Target is a simple box, thus we can ask its (DX,DY,DZ) half-dimensions:

In [5]:
dx = volTarget.GetShape().GetDX()
dy = volTarget.GetShape().GetDY()
dz = volTarget.GetShape().GetDZ()
print ("Our target has half-dimensions: ({:.1f},{:.1f},{:.1f}) cm".format(dx,dy,dz))

Our target has half-dimensions: (20.5,20.5,27.5) cm


Another example: often we need the Weight of a volume to predict the number of interactions in it, given the cross section. Let us ask for this quantity:

In [6]:
volTarget.Weight()

864.2794260000016

Info in <TGeoManager::Weight>: Computing analytically weight of volTarget
Info in <TGeoManager::Weight>: Computed weight: 864.279426 [kg]



As you notice from the printout, the weight is not a member of the volume, which can automatically be called from the class. It is a computation, by looping on daughters and getting density, more information can be found on TGeoVolume reference: https://root.cern.ch/doc/master/classTGeoVolume.html#a4fe233342614a24c755c34edaca5bbd2, along with all the other functions you can use.

For example, we can ask for shape and material properties with a nice printout:

In [7]:
volTarget.InspectShape()

*** Shape TargetBox: TGeoBBox ***
    dX =    20.50000
    dY =    20.50000
    dZ =    27.48875
    origin: x=    0.00000 y=    0.00000 z=    0.00000


In [8]:
volTarget.InspectMaterial()

Mixture air    Aeff=14.8329 Zeff=7.385 rho=0.001205 radlen=30245.4 intlen=71036.3 index=3
   Element #0 : N  Z=  7.00 A= 14.01 w= 0.755
   Element #1 : O  Z=  8.00 A= 16.00 w= 0.231
   Element #2 : AR  Z= 18.00 A= 39.95 w= 0.014
